In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
start_date = '2024-04-17' # match this with filename
end_date = '2024-04-17' # match this with filename
wt = pd.read_csv(f'G:/My Drive/NextGen/chat_history/chat_history_{start_date}_{end_date}.csv')

In [3]:
# dataframe cleaning
wt['created_at'] = wt['created_at'].apply(pd.to_datetime).dt.tz_localize(None)
wt.dropna(subset='text_messages', inplace=True)
wt.sort_values(['room_id', 'created_at'], inplace=True)
wt.reset_index(drop=True, inplace=True)

# save clean dataframe
# wt.to_excel(f'G:/My Drive/NextGen/wt_recap/chat history {start_date} to {end_date}.xlsx', index=False)

In [5]:
# Header mapping dict
daftar_gejala = {'Pemantauan Kesehatan Kehamilan':'[NextGen] pesan awal', 'Pemantauan Kesehatan Pasca Vaksinasi':'[PregVax] pesan awal',
                 'Gejala 1':'mual dan/atau muntah', 'Gejala 2':'tubuh lemas dan/atau mudah lelah', 'Gejala 3':'perut kram',
                 'Gejala 4':'sakit kepala', 'Gejala 5':'sesak napas', 'Gejala lain':'[NextGen] gejala lain / [PregVax] gejala',
                 'Seberapa parah':'tingkat keparahan', 'Tes PCR atau Antigen?':'tes COVID-19',
                 'pengobatan':'pengobatan'}

# Initialize an empty DataFrame to store processed customer responses
wt_repl = pd.DataFrame()

# Iterate through each symptom in the dictionary 'daftar_gejala'
for gejala in list(daftar_gejala.keys()):
    # Iterate through unique room IDs
    for room in wt['room_id'].unique():
        # Find messages sent by the system containing the current symptom and extract subsequent customer responses
        repl = wt[wt['room_id'] == room][['room_id', 'sender_type', 'text_messages']].shift(-1)[(wt['sender_type']=='system') & (wt['text_messages'].str.contains(gejala))].rename(columns={'text_messages':daftar_gejala[gejala]})
        # Store the customer responses in the DataFrame
        wt_repl = pd.concat([wt_repl, repl[repl['sender_type']=='customer']])
                
# Calculate the mode (most frequent) response for each room
wt_repl = wt_repl.drop(columns='sender_type').groupby('room_id', as_index=False).apply(lambda group: group.mode().iloc[0])
# Merge the processed DataFrame with room and customer information
wt_repl = wt[['room_id', 'customer_id', 'customer_name']].dropna().drop_duplicates().merge(wt_repl, how='left', on='room_id')

In [6]:
# save processed dataframe
wt_repl.to_excel(f'G:/My Drive/NextGen/wt_recap/wellness tracking {start_date} to {end_date}.xlsx', index=False)